# Route ETA Calculator with Ban Area Restrictions
## Install Required Packages (run once)

In [21]:
# Uncomment and run this cell if packages are not installed
# !pip install pandas openrouteservice python-dateutil folium

## Import Libraries

In [22]:
import os
import pandas as pd
import openrouteservice
from datetime import datetime, timedelta, time as dt_time
from dateutil import tz
import math
import folium
from IPython.display import display, HTML

## Configuration Parameters (EDIT THESE VALUES)

In [23]:
# =============================================================================
# CONFIGURATION - EDIT THESE VALUES FOR YOUR TRIP
# =============================================================================

# Trip coordinates
START_LAT = 24.7136
START_LON = 46.6753
END_LAT = 21.4225
END_LON = 39.8262

# Trip start time (ISO format with timezone)
START_DATETIME = "2025-07-02T23:31:50+03:00"

# OpenRouteService API key (get from https://openrouteservice.org/)
ORS_API_KEY = "your_api_key_here"  # Replace with your actual API key
# Alternatively, set as environment variable: os.getenv("ORS_API_KEY")

# Ban areas CSV file
BAN_CSV = "ban_times.csv"

# Ban area radius in kilometers
BAN_RADIUS_KM = 10

# =============================================================================

## Constants and Helper Functions

In [24]:
# Saudi Arabia timezone
SAUDI_TZ = tz.gettz('Asia/Riyadh')

# Haversine function to compute distance between two lat/lon points in km
def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # Earth radius in km
    phi1, phi2 = math.radians(lat1), math.radians(lat2)
    dphi = math.radians(lat2 - lat1)
    dlambda = math.radians(lon2 - lon1)
    a = math.sin(dphi / 2) ** 2 + math.cos(phi1) * math.cos(phi2) * math.sin(dlambda / 2) ** 2
    return 2 * R * math.asin(math.sqrt(a))

# Parse time string (e.g., '6:00') into datetime.time
def parse_time(tstr):
    h, m = map(int, tstr.split(":"))
    return dt_time(h, m)

# Given a weekday and ban row, get ban start and end datetime objects for the trip date
def get_ban_window(trip_date, ban_row):
    # trip_date: datetime.date
    start_time = parse_time(str(ban_row['Time_Start']))
    end_time = parse_time(str(ban_row['Time_End']))
    start_dt = datetime.combine(trip_date, start_time, tzinfo=SAUDI_TZ)
    end_dt = datetime.combine(trip_date, end_time, tzinfo=SAUDI_TZ)
    # Handle overnight ban windows (e.g., 23:00 to 01:00)
    if end_dt <= start_dt:
        end_dt += timedelta(days=1)
    return start_dt, end_dt

# Check if a point is within BAN_RADIUS_KM of a ban area
def point_in_ban_area(lat, lon, ban_lat, ban_lon, radius_km=BAN_RADIUS_KM):
    return haversine(lat, lon, ban_lat, ban_lon) <= radius_km

def load_ban_areas(csv_path):
    """Load ban areas from CSV into a DataFrame."""
    try:
        df = pd.read_csv(csv_path)
        return df
    except FileNotFoundError:
        print(f"Warning: {csv_path} not found. No ban areas will be applied.")
        return pd.DataFrame()

print("✓ Helper functions loaded successfully")

✓ Helper functions loaded successfully


## Load Ban Areas and Initialize Trip

In [25]:
# Load ban areas
ban_df = load_ban_areas(BAN_CSV)
print(f"Loaded {len(ban_df)} ban area records")

if not ban_df.empty:
    print("\nBan areas preview:")
    display(ban_df.head())
else:
    print("No ban areas loaded - continuing without restrictions")

# Parse trip start datetime
start_dt = datetime.fromisoformat(START_DATETIME)
if start_dt.tzinfo is None:
    start_dt = start_dt.replace(tzinfo=SAUDI_TZ)
else:
    start_dt = start_dt.astimezone(SAUDI_TZ)

trip_date = start_dt.date()
trip_weekday = start_dt.strftime('%A')

print(f"\nTrip Details:")
print(f"Start: {start_dt.strftime('%Y-%m-%d %H:%M:%S %Z')}")
print(f"Day of week: {trip_weekday}")
print(f"From: ({START_LAT:.4f}, {START_LON:.4f})")
print(f"To: ({END_LAT:.4f}, {END_LON:.4f})")

Loaded 38 ban area records

Ban areas preview:


,Country,City,Day_of_Week,Time_Start,Time_End,Latitude,Longitude
0,Saudi Arabia,Jeddah,Sunday,6:00,9:00,21.4858,39.1925
1,Saudi Arabia,Jeddah,Sunday,12:00,22:00,21.4858,39.1925
2,Saudi Arabia,Jeddah,Monday,6:00,9:00,21.4858,39.1925
3,Saudi Arabia,Jeddah,Monday,12:00,22:00,21.4858,39.1925
4,Saudi Arabia,Jeddah,Tuesday,6:00,9:00,21.4858,39.1925



Trip Details:
Start: 2025-07-02 23:31:50 +03
Day of week: Wednesday
From: (24.7136, 46.6753)
To: (21.4225, 39.8262)


## Fetch Route from OpenRouteService

In [26]:
# Prepare OpenRouteService client
if ORS_API_KEY == "5b3ce3597851110001cf62483a1b4539f1a94e73b7ebee0c67f14d18":
    print("⚠️  Please set your OpenRouteService API key in the configuration cell above")
    print("Get your free API key from: https://openrouteservice.org/")
else:
    try:
        client = openrouteservice.Client(key=ORS_API_KEY)
        coords = [(START_LON, START_LAT), (END_LON, END_LAT)]
        
        print("Fetching route from OpenRouteService...")
        route = client.directions(coords, profile='driving-car', format='geojson', instructions=False)
        
        geometry = route['features'][0]['geometry']['coordinates']  # list of [lon, lat]
        segments = geometry
        
        # Get total route distance and duration
        summary = route['features'][0]['properties']['summary']
        total_distance = summary['distance']  # meters
        total_duration = summary['duration']  # seconds
        
        print(f"✓ Route fetched successfully")
        print(f"Total distance: {total_distance/1000:.1f} km")
        print(f"Estimated duration: {total_duration/3600:.1f} hours")
        
    except Exception as e:
        print(f"❌ Error fetching route: {e}")
        route = None

Fetching route from OpenRouteService...
❌ Error fetching route: 403 ({'error': 'Access to this API has been disallowed'})


## Calculate ETA with Ban Area Restrictions

In [27]:
if route is not None:
    # Prepare ban windows for the trip day
    ban_windows = []
    if not ban_df.empty:
        for _, row in ban_df.iterrows():
            if row['Day_of_Week'] == trip_weekday:
                ban_lat, ban_lon = float(row['Latitude']), float(row['Longitude'])
                start_ban, end_ban = get_ban_window(trip_date, row)
                ban_windows.append({
                    'city': row['City'],
                    'start': start_ban,
                    'end': end_ban,
                    'lat': ban_lat,
                    'lon': ban_lon
                })
    
    print(f"Active ban windows for {trip_weekday}: {len(ban_windows)}")
    
    # Walk along the route, checking for ban area entry
    delays = []
    current_time = start_dt
    last_point = segments[0]
    distance_travelled = 0
    
    for i in range(1, len(segments)):
        p1 = last_point
        p2 = segments[i]
        seg_dist = haversine(p1[1], p1[0], p2[1], p2[0])  # in km
        seg_time = timedelta(seconds=(seg_dist * 1000) / total_distance * total_duration)  # proportional
        eta_to_seg = current_time + seg_time
        
        # Check each ban area
        for ban in ban_windows:
            # If segment enters ban area
            if point_in_ban_area(p2[1], p2[0], ban['lat'], ban['lon']):
                # If ETA falls within ban window, add wait
                if ban['start'] <= eta_to_seg <= ban['end']:
                    wait = ban['end'] - eta_to_seg
                    current_time = ban['end']
                    delays.append({
                        'city': ban['city'],
                        'wait': wait,
                        'ban_start': ban['start'],
                        'ban_end': ban['end'],
                        'eta_at_ban': eta_to_seg,
                        'lat': ban['lat'],
                        'lon': ban['lon'],
                        'stop_lat': p2[1],
                        'stop_lon': p2[0]
                    })
                    # Only delay once per ban area
                    ban_windows.remove(ban)
                    break
        
        current_time += seg_time
        last_point = p2
        distance_travelled += seg_dist
    
    print(f"\n✓ Route analysis complete")
    print(f"Delays encountered: {len(delays)}")

## Display Results

In [28]:
if route is not None:
    # Output results
    print(f"\n{'='*50}")
    print(f"ESTIMATED ETA: {current_time.strftime('%Y-%m-%d %H:%M:%S %Z')}")
    print(f"{'='*50}")
    
    if delays:
        print("\n🚫 DELAYS ENCOUNTERED:")
        print("-" * 40)
        for d in delays:
            # Round up wait to nearest minute
            wait_minutes = int((d['wait'].total_seconds() + 59) // 60)
            print(f"• City: {d['city']}")
            print(f"  Wait: {wait_minutes} minutes")
            print(f"  ETA at Ban: {d['eta_at_ban'].strftime('%H:%M')}")
            print(f"  Ban Window: {d['ban_start'].strftime('%H:%M')} to {d['ban_end'].strftime('%H:%M')}")
            print()
    else:
        print("\n✅ No ban area delays encountered.")
    
    # Print detailed schedule
    print(f"\n{'='*50}")
    print("DETAILED TRIP SCHEDULE")
    print(f"{'='*50}")
    print(f"START:   {start_dt.strftime('%Y-%m-%d %H:%M')} at ({START_LAT:.4f}, {START_LON:.4f})")
    
    last_time = start_dt
    last_lat = START_LAT
    last_lon = START_LON
    
    for d in delays:
        wait_minutes = int((d['wait'].total_seconds() + 59) // 60)
        print(f"BAN ARR: {d['eta_at_ban'].strftime('%Y-%m-%d %H:%M')} at ({d['stop_lat']:.4f}, {d['stop_lon']:.4f}) [{d['city']}] (wait {wait_minutes} min)")
        print(f"BAN DEP: {(d['eta_at_ban'] + d['wait']).strftime('%Y-%m-%d %H:%M')} at ({d['stop_lat']:.4f}, {d['stop_lon']:.4f}) [{d['city']}]")
        last_time = d['eta_at_ban'] + d['wait']
        last_lat = d['stop_lat']
        last_lon = d['stop_lon']
    
    print(f"END:     {current_time.strftime('%Y-%m-%d %H:%M')} at ({END_LAT:.4f}, {END_LON:.4f})")

## Generate Interactive Map

In [29]:
if route is not None:
    # Center map between start and end
    mid_lat = (START_LAT + END_LAT) / 2
    mid_lon = (START_LON + END_LON) / 2
    m = folium.Map(location=[mid_lat, mid_lon], zoom_start=6)
    
    # Draw route polyline
    folium.PolyLine(
        [(lat, lon) for lon, lat in segments], 
        color="blue", 
        weight=5, 
        opacity=0.7,
        tooltip="Route"
    ).add_to(m)
    
    # Mark start and end
    folium.Marker(
        [START_LAT, START_LON],
        popup=f"Start ({START_LAT:.4f}, {START_LON:.4f})<br>{start_dt.strftime('%Y-%m-%d %H:%M')}",
        icon=folium.Icon(color="green", icon="play")
    ).add_to(m)
    
    folium.Marker(
        [END_LAT, END_LON],
        popup=f"End ({END_LAT:.4f}, {END_LON:.4f})<br>{current_time.strftime('%Y-%m-%d %H:%M')}",
        icon=folium.Icon(color="red", icon="stop")
    ).add_to(m)
    
    # Mark ban stops at the actual stop location
    for d in delays:
        wait_minutes = int((d['wait'].total_seconds() + 59) // 60)
        departure_time = (d['eta_at_ban'] + d['wait']).strftime('%Y-%m-%d %H:%M')
        folium.CircleMarker(
            location=[d['stop_lat'], d['stop_lon']],
            radius=14,
            color="orange",
            fill=True,
            fill_color="red",
            fill_opacity=0.95,
            tooltip=f"Ban Stop: {d['city']}",
            popup=folium.Popup(
                f"<b>Ban Stop: {d['city']}</b><br>"
                f"Arrival: {d['eta_at_ban'].strftime('%Y-%m-%d %H:%M')}<br>"
                f"Departure: {departure_time}<br>"
                f"Wait: {wait_minutes} min<br>"
                f"Ban Window: {d['ban_start'].strftime('%H:%M')} - {d['ban_end'].strftime('%H:%M')}",
                max_width=300
            )
        ).add_to(m)
    
    # Save and display map
    map_file = "route_map.html"
    m.save(map_file)
    print(f"✓ Interactive map saved as {map_file}")
    print("✓ Map will be displayed below:")
    
    # Display map in notebook
    display(m)
else:
    print("❌ Cannot generate map - route data not available")

❌ Cannot generate map - route data not available


## Summary Statistics (Optional)

In [30]:
if route is not None:
    total_wait_time = sum(d['wait'].total_seconds() for d in delays) / 60  # minutes
    original_duration = total_duration / 60  # minutes
    final_duration = (current_time - start_dt).total_seconds() / 60  # minutes
    
    print(f"\n{'='*50}")
    print("TRIP SUMMARY STATISTICS")
    print(f"{'='*50}")
    print(f"Original estimated duration: {original_duration:.0f} minutes ({original_duration/60:.1f} hours)")
    print(f"Total wait time due to bans: {total_wait_time:.0f} minutes ({total_wait_time/60:.1f} hours)")
    print(f"Final trip duration: {final_duration:.0f} minutes ({final_duration/60:.1f} hours)")
    print(f"Delay percentage: {(total_wait_time/original_duration*100):.1f}%")
    print(f"Distance: {total_distance/1000:.1f} km")
    print(f"Number of ban stops: {len(delays)}")